In [8]:
import json 
import numpy as nphello
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tkinter import *
from tkinter import ttk

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle
import time

with open("intents.json") as file:
    data = json.load(file)


def chatbox():

    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    def chatLog():
        text = textBox.get()
        inp = text

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])


        textBox.delete(0, 'end')
        chat.configure(state='normal')
        chat.insert('end', 'User: ' + text + '\n')
        for i in data['intents']:
            if i['tag'] == tag:
                chat.insert('end', 'Moviebot: ' + np.random.choice(i['responses']) + '\n')
        if tag == "genres":
            chat.insert('end', 'Moviebot: ' + "Got one for you! How about.. '" + webscraper(inp) + "'?" + '\n')
            genre = inp
        if tag == "another":
            chat.insert('end', 'Moviebot: ' + webscraper(genre) + '\n')
        chat.configure(state='disabled')


    root = Tk()
    chatBox = Scrollbar(root)
    chat = Text(root, wrap='word', state='disabled', width=50,
                yscrollcommand=chatBox.set)
    chatBox.configure(command=chat.yview)

    chat.grid(row=0, columnspan=2, sticky='ewns')
    chatBox.grid(row=0, column=2, sticky='ns')
    Label(root, text="Input: ").grid(row=1, column=0)

    textBox = Entry(root, bd=0, width=40, bg="lightblue")
    textBox.grid(row=1, column=1)

    Button(root, text="Send", command=chatLog).grid(row=2, columnspan=2)
    root.mainloop()

chatbox()
